<a href="https://colab.research.google.com/github/nazilaeshghi/from-rep/blob/main/HW3_Zoomcamp2024_MachineLearning_Classification_NazilaEshghi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework
Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

Dataset
In this homework, we will use the Bank Marketing dataset. Download it from here.

Or you can do it with wget:

wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

We need to take bank/bank-full.csv file from the downloaded zip-file.
In this dataset our desired target for classification task will be y variable - has the client subscribed a term deposit or not.

In [23]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [24]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-15 05:54:52--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip.1’

bank+marketing.zip.     [ <=>                ] 999.85K  --.-KB/s    in 0.1s    

2024-10-15 05:54:52 (9.44 MB/s) - ‘bank+marketing.zip.1’ saved [1023843]



In [25]:
# # Load the dataframe.
# with zipfile.ZipFile('bank+marketing.zip', 'r') as zip_ref:
#     # Extract the CSV file to a temporary location or read directly into memory
#     # Here, we read directly using TextIOWrapper
#     # The file path was incorrect, it should be 'bank/bank-full.csv'
#     with zip_ref.open('bank+marketing/bank/bank-full.csv') as f:  # Updated file path within the zip
#         df = pd.read_csv(f, sep=';') # Added sep=';' as the file is semicolon-separated

# # Now you can manipulate the columns.
# df.columns = df.columns.str.lower().str.replace(' ', '_')

In [26]:
# Load the dataframe.
df = pd.read_csv('bank-full.csv', sep= ';')

# Now you can manipulate the columns.
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [27]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [28]:
df.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
balance,int64
housing,object
loan,object
contact,object
day,int64


### Features

For the rest of the homework, you'll need to use only these columns:

* `age`,
* `job`,
* `marital`,
* `education`,
* `balance`,
* `housing`,
* `contact`,
* `day`,
* `month`,
* `duration`,
* `campaign`,
* `pdays`,
* `previous`,
* `poutcome`,
* `y`

In [29]:
# Select desired columns
base  = ['age', 'job', 'marital', 'education', 'default', 'balance',
                   'housing', 'loan', 'contact', 'day', 'month', 'duration',
                   'campaign', 'pdays', 'previous', 'poutcome', 'y']
data = df[base]
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


### Data preparation

* Select only the features from above.
* Check if the missing values are presented in the features.

In [30]:
print(data.isnull().sum())

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


In [31]:
# # Replace 'yes' and 'no' in the 'y' column with 1 and 0
# df['y'] = df['y'].map({'yes': 1, 'no': 0})

# # Separate features (X) and target (y)
# X = df.drop('y', axis=1)  # Features (all columns except 'y')
# y = df['y']              # Target variable ('y')

# # Split into train/val/test sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train, 40% temp
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test


### Question 1

What is the most frequent observation (mode) for the column `education`?

- `unknown`
- `primary`
- `secondary`
- `tertiary`

In [32]:
education_mode = df['education'].mode()[0]  # [0] to get the first element (mode)

print("The most frequent observation (mode) for education is:", education_mode)

The most frequent observation (mode) for education is: secondary


### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `age` and `balance`
- `day` and `campaign`
- `day` and `pdays`
- `pdays` and `previous`


In [33]:
# Replace 'yes' and 'no' in the 'y' column with 1 and 0
df['y'] = df['y'].map({'yes': 1, 'no': 0})

# Separate features (X) and target (y)
X = df.drop('y', axis=1)  # Features (all columns except 'y')
y = df['y']              # Target variable ('y')

# Split into train/val/test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train, 40% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# Select numerical features
numerical_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [34]:
# Calculate the correlation matrix on the training set
correlation_matrix = X_train[numerical_features].corr()

In [35]:
# Find the two features with the biggest absolute correlation (excluding self-correlation)
correlation_matrix_abs = correlation_matrix.abs()
np.fill_diagonal(correlation_matrix_abs.values, 0)  # Set diagonal to 0 to exclude self-correlation
max_value = correlation_matrix_abs.max().max()  # Get the maximum absolute correlation value
row_max, col_max = correlation_matrix_abs[correlation_matrix_abs == max_value].stack().idxmax()

print("Correlation matrix:")
print(correlation_matrix)
print("\nThe two features with the biggest correlation are:", row_max, "and", col_max)

Correlation matrix:
               age   balance       day  duration  campaign     pdays  previous
age       1.000000  0.100432 -0.008738 -0.009044  0.003054 -0.018360  0.010721
balance   0.100432  1.000000  0.000402  0.025567 -0.025809  0.007944  0.025133
day      -0.008738  0.000402  1.000000 -0.020818  0.161739 -0.095842 -0.067717
duration -0.009044  0.025567 -0.020818  1.000000 -0.083317 -0.000950 -0.001725
campaign  0.003054 -0.025809  0.161739 -0.083317  1.000000 -0.088432 -0.039348
pdays    -0.018360  0.007944 -0.095842 -0.000950 -0.088432  1.000000  0.551935
previous  0.010721  0.025133 -0.067717 -0.001725 -0.039348  0.551935  1.000000

The two features with the biggest correlation are: pdays and previous


### Target encoding

* Now we want to encode the `y` variable.
* Let's replace the values `yes`/`no` with `1`/`0`.

In [36]:
# Assuming 'df' is your DataFrame
df['y'] = df['y'].map({'yes': 1, 'no': 0})





### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.

In [37]:
# Split into train/val/test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train, 40% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

print("Train set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])

Train set size: 27126
Validation set size: 9042
Test set size: 9043


### Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `contact`
- `education`
- `housing`
- `poutcome`

In [38]:
# Select categorical features
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Calculate mutual information scores
mi_scores = {}
for feature in categorical_features:
    mi_score = mutual_info_score(X_train[feature], y_train)
    mi_scores[feature] = round(mi_score, 2)

# Find the feature with the biggest mutual information score
max_mi_feature = max(mi_scores, key=mi_scores.get)

print("Mutual Information Scores:")
for feature, score in mi_scores.items():
    print(f"{feature}: {score}")

print("\nThe variable with the biggest mutual information score is:", max_mi_feature)

Mutual Information Scores:
job: 0.01
marital: 0.0
education: 0.0
default: 0.0
housing: 0.01
loan: 0.0
contact: 0.01
month: 0.02
poutcome: 0.03

The variable with the biggest mutual information score is: poutcome




### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.6
- 0.7
- 0.8
- 0.9


In [40]:
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numerical_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

train_dicts = X_train[categorical_features + numerical_features].to_dict(orient='records')

dv = DictVectorizer(sparse=False) # Use sparse=False instead of sparse_output=False
X_train_encoded = dv.fit_transform(train_dicts)

# Initialize and train the logistic regression model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [41]:
# Transform the validation data
val_dicts = X_val[categorical_features + numerical_features].to_dict(orient='records')
X_val_encoded = dv.transform(val_dicts)  # Use transform, not fit_transform, for validation data

# Make predictions on the validation set
y_pred = model.predict(X_val_encoded)

# Calculate accuracy and round to 2 decimal places
accuracy = accuracy_score(y_val, y_pred)
rounded_accuracy = round(accuracy, 2)

print(f"Accuracy on validation set: {rounded_accuracy}")


Accuracy on validation set: 0.9


**Question 5**

Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.</b>
Which of following feature has the smallest difference?

* age
* balance
* marital
* previous




In [43]:
# Train the model with all features and calculate accuracy
val_dicts = X_val[categorical_features + numerical_features].to_dict(orient='records')
X_val_encoded = dv.transform(val_dicts)
y_pred = model.predict(X_val_encoded)
original_accuracy = accuracy_score(y_val, y_pred)

# Feature elimination loop for specified features
feature_importance = {}
for feature in ['age', 'balance', 'marital', 'previous']:
    # Create a list of features without the current feature
    features_subset = [f for f in categorical_features + numerical_features if f != feature]

    # Train a model without the current feature
    train_dicts_subset = X_train[features_subset].to_dict(orient='records')
    dv_subset = DictVectorizer(sparse=False)
    X_train_encoded_subset = dv_subset.fit_transform(train_dicts_subset)
    model_subset = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_subset.fit(X_train_encoded_subset, y_train)

    # Calculate accuracy on validation set without the current feature
    val_dicts_subset = X_val[features_subset].to_dict(orient='records')
    X_val_encoded_subset = dv_subset.transform(val_dicts_subset)
    y_pred_subset = model_subset.predict(X_val_encoded_subset)
    accuracy_subset = accuracy_score(y_val, y_pred_subset)

    # Store the accuracy difference
    feature_importance[feature] = original_accuracy - accuracy_subset

# Find the feature with the smallest difference (least useful)
least_useful_feature = min(feature_importance, key=feature_importance.get)

print("Feature Importance (Accuracy Difference):")
for feature, importance in feature_importance.items():
    print(f"{feature}: {importance:.4f}")

print(f"\nThe least useful feature is: {least_useful_feature}")

Feature Importance (Accuracy Difference):
age: -0.0002
balance: 0.0000
marital: -0.0008
previous: 0.0002

The least useful feature is: marital


**Question 6** </b>
 Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
</b>Train models using all the features as in Q4.</b>
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.</b>
Which of these C leads to the best accuracy on the validation set?
</b>
* 0.01
* 0.1
* 1
* 10
* 100
Note: If there are multiple options, select the smallest C.



In [44]:
# Define the C values to try
C_values = [0.01, 0.1, 1, 10, 100]

# Dictionary to store accuracy for each C value
accuracy_scores = {}

# Loop through C values and train models
for C in C_values:
    # Initialize and train the model
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, y_train)

    # Calculate accuracy on validation set
    val_dicts = X_val[categorical_features + numerical_features].to_dict(orient='records')
    X_val_encoded = dv.transform(val_dicts)
    y_pred = model.predict(X_val_encoded)
    accuracy = accuracy_score(y_val, y_pred)

    # Store accuracy rounded to 3 decimal places
    accuracy_scores[C] = round(accuracy, 3)

# Find the C value with the best accuracy
best_C = max(accuracy_scores, key=accuracy_scores.get)

print("Accuracy Scores for Different C Values:")
for C, accuracy in accuracy_scores.items():
    print(f"C = {C}: Accuracy = {accuracy}")

print(f"\nThe best C value is: {best_C}")

Accuracy Scores for Different C Values:
C = 0.01: Accuracy = 0.898
C = 0.1: Accuracy = 0.901
C = 1: Accuracy = 0.9
C = 10: Accuracy = 0.901
C = 100: Accuracy = 0.9

The best C value is: 0.1
